# Caderno 12 - Compila todas as métricas em 6 tabelas.

As tabelas são as combinações dos conjuntos de queries (3 conjuntos) e k = [5, 10, 20].

## 1. Pastas com os resultados, nomes dos arquivos de índice. docs/queries/qrels

In [1]:
import pandas as pd

PASTA_DADOS = './dados/'
# A pasta dos JURIS aqui não é a pasta original, e sim o resultado do caderno 1 (os documentos já estão filtrados)
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Cenário atual
PASTA_RESULTADO_PESQUISA_SOLR = f'{PASTA_DADOS}outputs/2_pesquisa_queries_na_base_atual/'

# Cenário BM25 padrão, com enunciado único e com enunciado dobrado
INDICE_BM25_PADRAO = f'{PASTA_DADOS}/outputs/4_metricas_bm25_padrao/indice_js_enunciado_e_excerto.pickle'
INDICE_BM25_PADRAO_ENUNCIADO_DOBRADO = f'{PASTA_DADOS}/outputs/4_metricas_bm25_padrao/indice_js_enunciado_2x_e_excerto.pickle'
# Cenário BM25 padrão com DOC2QUERY
INDICE_BM25_COM_DOC2QUERY_1QUERY = f'{PASTA_DADOS}/outputs/6_metricas_bm25_com_doc2query/indice_js_enunciado_e_excerto_e_doc2query_apenas_uma_query.pickle'
INDICE_BM25_COM_DOC2QUERY_5QUERIES = f'{PASTA_DADOS}/outputs/6_metricas_bm25_com_doc2query/indice_js_enunciado_e_excerto_e_doc2query.pickle'
# Cenário BM25 padrão com reescrita de enunciado
INDICE_BM25_COM_REESCRITA_ENUNCIADO_GPT = f'{PASTA_DADOS}/outputs/8_metricas_bm25_com_reescrita_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_enunciado_reescrito_gpt.pickle'
INDICE_BM25_COM_REESCRITA_ENUNCIADO_LLAMA = f'{PASTA_DADOS}/outputs/8_metricas_bm25_com_reescrita_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_enunciado_reescrito_llama.pickle'
# Cenário BM25 padrão com sinônimos do enunciado
INDICE_BM25_COM_SINONIMOS_ENUNCIADO_GPT = f'{PASTA_DADOS}/outputs/10_metricas_bm25_com_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_sinonimos_enunciado_gpt.pickle'
INDICE_BM25_COM_SINONIMOS_ENUNCIADO_LLAMA = f'{PASTA_DADOS}/outputs/10_metricas_bm25_com_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_sinonimos_enunciado_llama.pickle'
# Cenário BM25 padrão com DOC2QUERY (5 queries) e sinônimos do enunciado
INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_GPT = f'{PASTA_DADOS}/outputs/11_metricas_bm25_com_doc2query_e_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_doc2query_5q_e_gpt.pickle'
INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_LLAMA = f'{PASTA_DADOS}/outputs/11_metricas_bm25_com_doc2query_e_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_doc2query_5q_e_llama.pickle'


# Carrega o resultado da pesquisa do Solr
resultado_pesq_solr = pd.read_csv(f'{PASTA_RESULTADO_PESQUISA_SOLR}resultado_solr_pesquisa_original.csv', sep='|')
resultado_pesq_solr = resultado_pesq_solr[resultado_pesq_solr.ENGINE == 'SOLR_PESQUISA_ORIGINAL_selectSwanSynonym']

In [2]:
# Carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query_tratado.csv', sep='|')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')

    return doc, query, qrel

docs, queries, qrels = carrega_juris_tcu()

## 2. Cria índices e buscadores

In [3]:
from bm25 import IndiceInvertido, BM25, tokenizador_pt_remove_html

# Lista ordenada para exibição
lista_nome_buscadores = ["atual",
                       "bm25_padrao",
                       "bm25_padrao_enunciado_dobrado",
                       "doc2query_1query",
                       "doc2query_5queries",
                       "reescrita_enunciado_gpt",
                       "reescrita_enunciado_llama",
                       "sinonimos_enunciado_gpt",
                       "sinonimos_enunciado_llama",
                       "doc2query_5queries_sinonimos_gpt",
                       "doc2query_5queries_sinonimos_llama"]

mapa_nome_arquivos_indices = {
    "bm25_padrao": INDICE_BM25_PADRAO,
    "bm25_padrao_enunciado_dobrado": INDICE_BM25_PADRAO_ENUNCIADO_DOBRADO,
    "doc2query_1query": INDICE_BM25_COM_DOC2QUERY_1QUERY,
    "doc2query_5queries": INDICE_BM25_COM_DOC2QUERY_5QUERIES,
    "reescrita_enunciado_gpt": INDICE_BM25_COM_REESCRITA_ENUNCIADO_GPT,
    "reescrita_enunciado_llama": INDICE_BM25_COM_REESCRITA_ENUNCIADO_LLAMA,
    "sinonimos_enunciado_gpt": INDICE_BM25_COM_SINONIMOS_ENUNCIADO_GPT,
    "sinonimos_enunciado_llama": INDICE_BM25_COM_SINONIMOS_ENUNCIADO_LLAMA,
    "doc2query_5queries_sinonimos_gpt": INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_GPT,
    "doc2query_5queries_sinonimos_llama": INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_LLAMA
}

# Cria os IndiceInvertidos
mapa_indices = {nome: IndiceInvertido(tokenizador_pt_remove_html) for nome, _ in mapa_nome_arquivos_indices.items()}

# Carrega dos pickles
for nome, indice in mapa_indices.items():
    print(f'Carregando indice {nome} com o arquivo {mapa_nome_arquivos_indices[nome]}')
    indice.from_pickle(mapa_nome_arquivos_indices[nome])

# Cria os buscadores
# Sobre a escolha de k1 e b:
# https://www.elastic.co/pt/blog/practical-bm25-part-3-considerations-for-picking-b-and-k1-in-elasticsearch
mapa_buscadores = {nome: BM25(indice, k1=1.2, b=0.75, bias_idf=1) for nome, indice in mapa_indices.items()}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\caris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\caris\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Carregando indice bm25_padrao com o arquivo ./dados//outputs/4_metricas_bm25_padrao/indice_js_enunciado_e_excerto.pickle
Carregando indice bm25_padrao_enunciado_dobrado com o arquivo ./dados//outputs/4_metricas_bm25_padrao/indice_js_enunciado_2x_e_excerto.pickle
Carregando indice doc2query_1query com o arquivo ./dados//outputs/6_metricas_bm25_com_doc2query/indice_js_enunciado_e_excerto_e_doc2query_apenas_uma_query.pickle
Carregando indice doc2query_5queries com o arquivo ./dados//outputs/6_metricas_bm25_com_doc2query/indice_js_enunciado_e_excerto_e_doc2query.pickle
Carregando indice reescrita_enunciado_gpt com o arquivo ./dados//outputs/8_metricas_bm25_com_reescrita_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_enunciado_reescrito_gpt.pickle
Carregando indice reescrita_enunciado_llama com o arquivo ./dados//outputs/8_metricas_bm25_com_reescrita_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_enunciado_reescrito_llama.pickle
Carregando indice sinonimos_enunciado_gpt com o arqu

## 3. Extrai as métricas

In [4]:
from tqdm import tqdm

def pesquisar_usando_buscador(nome, buscador):
    col_resultado_query_key=[]
    col_resultado_doc_key=[]
    col_resultado_rank=[]
    
    for i, row in tqdm(queries.iterrows(), total=len(queries), desc=f'Pesquisando usando {nome}'):
        query_key = row.KEY
        query_text = row.TEXT
        resultados = buscador.pesquisar(query_text)
    
        primeiros_50_docs = [tupla_key_score[0] for tupla_key_score in resultados[:50]]
        queries_keys = [query_key] * len(primeiros_50_docs)
        ranking = list(range(1, len(primeiros_50_docs)+1))
    
        col_resultado_query_key.extend(queries_keys)
        col_resultado_doc_key.extend(primeiros_50_docs)
        col_resultado_rank.extend(ranking)
    
    df_resultados = pd.DataFrame({
        "QUERY_KEY": col_resultado_query_key,
        "DOC_KEY": col_resultado_doc_key,
        "RANK": col_resultado_rank,
    })
    return df_resultados

In [5]:
from metricas import metricas

# Pesquisa em todos os buscadores
mapa_resultados = {nome: pesquisar_usando_buscador(nome, buscador) for nome, buscador in mapa_buscadores.items()}

# Extrai as métricas
mapa_metricas = {nome: metricas(resultados, qrels, aproximacao_trec_eval=True) for nome, resultados in mapa_resultados.items()}
# Insere no mapa de métricas as métricas atuais
mapa_metricas['atual'] = metricas(resultado_pesq_solr, qrels, aproximacao_trec_eval=True)

Pesquisando usando sinonimos_enunciado_llama: 100%|██████████████████████████████████| 150/150 [00:02<00:00, 55.59it/s]
Pesquisando usando doc2query_5queries_sinonimos_gpt: 100%|███████████████████████████| 150/150 [00:02<00:00, 57.13it/s]
Pesquisando usando doc2query_5queries_sinonimos_llama: 100%|█████████████████████████| 150/150 [00:02<00:00, 56.66it/s]


## 4. Exibe as métricas

In [15]:
# Imprime as métricas para o conjunto de queries 1 (0:50), 2 (100:150), ou 3 (100:150) 
# e para um determinado k (foi gerado para k = 5, 10, 20 e 50.

def compara_metricas(con_query, k):
    # Acumula as métricas
    precisao = []
    recall = []
    mrr = []
    ndcg = []

    for nome in lista_nome_buscadores:
        estatisticas = mapa_metricas[nome][50*(con_query-1):50*(con_query)].describe()
        precisao.append(estatisticas.loc['mean', f'P@{k}'])
        recall.append(estatisticas.loc['mean', f'R@{k}'])
        mrr.append(estatisticas.loc['mean', f'MRR@{k}'])
        ndcg.append(estatisticas.loc['mean', f'nDCG@{k}'])

    df = pd.DataFrame({
        "Modelo": lista_nome_buscadores,
        f"P@{k}": precisao,
        f"R@{k}": recall,
        f"MRR@{k}": mrr,
        f"nDCG@{k}": ndcg
    })
    return df

def compara_metricas_todas_queries(k):
    # Acumula as métricas
    precisao = []
    recall = []
    mrr = []
    ndcg = []

    for nome in lista_nome_buscadores:
        estatisticas = mapa_metricas[nome].describe()
        precisao.append(estatisticas.loc['mean', f'P@{k}'])
        recall.append(estatisticas.loc['mean', f'R@{k}'])
        mrr.append(estatisticas.loc['mean', f'MRR@{k}'])
        ndcg.append(estatisticas.loc['mean', f'nDCG@{k}'])

    df = pd.DataFrame({
        "Modelo": lista_nome_buscadores,
        f"P@{k}": precisao,
        f"R@{k}": recall,
        f"MRR@{k}": mrr,
        f"nDCG@{k}": ndcg
    })
    return df

pd.set_option('display.precision', 4)

In [21]:
for con_query in [1, 2, 3]:
#    for k in [5, 10, 20]:
    for k in [10]:
        print(f'Resultados para conjunto de query {con_query} e k={k}')
        display(compara_metricas(con_query, k))

Resultados para conjunto de query 1 e k=10


,Modelo,P@10,R@10,MRR@10,nDCG@10
0,atual,0.260,0.2120,0.3963,0.2656
1,bm25_padrao,0.238,0.1966,0.5386,0.2753
2,bm25_padrao_enunciado_dobrado,0.286,0.2367,0.5352,0.3147
3,doc2query_1query,0.270,0.2230,0.5406,0.3042
4,doc2query_5queries,0.336,0.2739,0.6539,0.3851
5,reescrita_enunciado_gpt,0.262,0.2143,0.5196,0.2881
6,reescrita_enunciado_llama,0.262,0.2146,0.4579,0.2792
7,sinonimos_enunciado_gpt,0.268,0.2214,0.5507,0.3073
8,sinonimos_enunciado_llama,0.274,0.2266,0.5476,0.3084
9,doc2query_5queries_sinonimos_gpt,0.354,0.2882,0.6749,0.4041


Resultados para conjunto de query 2 e k=10


,Modelo,P@10,R@10,MRR@10,nDCG@10
0,atual,0.288,0.2396,0.8667,0.4579
1,bm25_padrao,0.378,0.3176,0.8665,0.5106
2,bm25_padrao_enunciado_dobrado,0.416,0.3488,0.9394,0.5625
3,doc2query_1query,0.380,0.3190,0.8965,0.5198
4,doc2query_5queries,0.402,0.3380,0.8816,0.5458
5,reescrita_enunciado_gpt,0.388,0.3258,0.8958,0.5291
6,reescrita_enunciado_llama,0.396,0.3325,0.8940,0.5323
7,sinonimos_enunciado_gpt,0.408,0.3428,0.8845,0.5435
8,sinonimos_enunciado_llama,0.394,0.3307,0.8735,0.5297
9,doc2query_5queries_sinonimos_gpt,0.424,0.3561,0.8925,0.5660


Resultados para conjunto de query 3 e k=10


,Modelo,P@10,R@10,MRR@10,nDCG@10
0,atual,0.018,0.0164,0.1100,0.0372
1,bm25_padrao,0.388,0.3451,0.9175,0.5328
2,bm25_padrao_enunciado_dobrado,0.418,0.3726,0.9425,0.5673
3,doc2query_1query,0.392,0.3485,0.9525,0.5435
4,doc2query_5queries,0.408,0.3616,0.9389,0.5561
5,reescrita_enunciado_gpt,0.406,0.3601,0.9425,0.5554
6,reescrita_enunciado_llama,0.412,0.3655,0.9550,0.5622
7,sinonimos_enunciado_gpt,0.406,0.3605,0.9153,0.5461
8,sinonimos_enunciado_llama,0.396,0.3524,0.9237,0.5409
9,doc2query_5queries_sinonimos_gpt,0.416,0.3686,0.9187,0.5574


In [14]:
for k in [5, 10, 20]:
    display(compara_metricas_todas_queries(k))

,Modelo,P@5,R@5,MRR@5,nDCG@5
0,atual,0.260000,0.107022,0.449556,0.293402
1,bm25_padrao,0.430667,0.184124,0.767444,0.485555
2,bm25_padrao_enunciado_dobrado,0.465333,0.199370,0.794889,0.526173
3,doc2query_1query,0.436000,0.185974,0.786889,0.498613
4,doc2query_5queries,0.490667,0.208572,0.818556,0.549379
5,reescrita_enunciado_gpt,0.445333,0.190616,0.776889,0.501491
6,reescrita_enunciado_llama,0.450667,0.192644,0.762222,0.501122
7,sinonimos_enunciado_gpt,0.453333,0.192987,0.777222,0.509126
8,sinonimos_enunciado_llama,0.450667,0.191946,0.776222,0.506839
9,doc2query_5queries_sinonimos_gpt,0.514667,0.217570,0.824444,0.567383


,Modelo,P@10,R@10,MRR@10,nDCG@10
0,atual,0.188667,0.155996,0.457648,0.253574
1,bm25_padrao,0.334667,0.286422,0.774209,0.439576
2,bm25_padrao_enunciado_dobrado,0.373333,0.319357,0.805693,0.481509
3,doc2query_1query,0.347333,0.296822,0.796524,0.455841
4,doc2query_5queries,0.382000,0.324532,0.824778,0.495695
5,reescrita_enunciado_gpt,0.352000,0.300076,0.785981,0.457516
6,reescrita_enunciado_llama,0.356667,0.304198,0.768960,0.457898
7,sinonimos_enunciado_gpt,0.360667,0.308208,0.783526,0.465645
8,sinonimos_enunciado_llama,0.354667,0.303254,0.781582,0.459675
9,doc2query_5queries_sinonimos_gpt,0.398000,0.337672,0.828675,0.509170


,Modelo,P@20,R@20,MRR@20,nDCG@20
0,atual,0.128000,0.213010,0.460331,0.274564
1,bm25_padrao,0.249667,0.425778,0.776184,0.500354
2,bm25_padrao_enunciado_dobrado,0.261000,0.443696,0.807802,0.530971
3,doc2query_1query,0.256667,0.436507,0.798481,0.516493
4,doc2query_5queries,0.272333,0.461141,0.826341,0.550478
5,reescrita_enunciado_gpt,0.254667,0.433614,0.788567,0.514332
6,reescrita_enunciado_llama,0.260667,0.444408,0.772000,0.517224
7,sinonimos_enunciado_gpt,0.258667,0.440389,0.785392,0.520481
8,sinonimos_enunciado_llama,0.256667,0.436925,0.782664,0.518057
9,doc2query_5queries_sinonimos_gpt,0.277000,0.469266,0.829230,0.560041
